<a href="https://colab.research.google.com/github/d-pujara/bubble.ai/blob/main/news_aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
from newsapi import NewsApiClient
import pandas as pd
from datetime import datetime

# Initialize the News API client
newsapi = NewsApiClient(api_key='74d7df82b0314dad9eb78ddd579748e3')

# Function to extract necessary information from an article
def extract_article_info(article):
    return {
        'source': article['source']['name'],
        'author': article.get('author'),
        'description': article.get('description'),
        'title': article.get('title'),
        'url': article.get('url'),
        'urlToImage': article.get('urlToImage'),
        'publishedAt': article.get('publishedAt'),
        'content': article.get('content')
    }

# Function to fetch the latest articles for a given topic
def fetch_latest_news(topic):
    response = newsapi.get_everything(q=topic,
                                      language='en',
                                      sort_by='publishedAt',
                                      page=1)
    articles = response['articles']
    return [extract_article_info(article) for article in articles]

# List of topics to fetch news for
topics = ['technology', 'finance', 'health', 'sports']  # Example topics

iteration_count = 0
max_iterations = 10  # Set a maximum number of iterations if needed

while iteration_count < max_iterations:
    all_articles_data = []
    for topic in topics:
        latest_articles = fetch_latest_news(topic)
        all_articles_data.extend(latest_articles)

    # Convert the articles data to a DataFrame
    df_articles = pd.DataFrame(all_articles_data)

    # Timestamp for the filename
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'news_articles_{timestamp}.csv'

    # Save to CSV
    df_articles.to_csv(filename, index=False)

    print(f"Saved articles to {filename}")

    # Increment the iteration count and wait before the next iteration
    iteration_count += 1
    time.sleep(60)  # Wait for 1 hour (3600 seconds)

print("Finished fetching articles.")

ModuleNotFoundError: ignored

In [ ]:
import csv
with open('news_articles_20231206_063802.csv', 'r') as f:
  reader = csv.reader(f)
  for row in reader:
    print(row)

In [ ]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import nltk

# Load the CSV file
df = pd.read_csv('news_articles_20231206_063802.csv')

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Preprocess the text data
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stop_words:
            result.append(token)
    return result

# Apply preprocessing to the description column
processed_docs = df['description'].map(preprocess)

# Create a dictionary and corpus needed for LDA model
dictionary = corpora.Dictionary(processed_docs)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=dictionary,
                                       num_topics=10,  # You can choose the number of topics
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

# Function to get the dominant topic for each description
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=processed_docs):
    topic_list = []
    for row in range(len(df)):
        topic, _ = max(ldamodel[corpus[row]], key=lambda x: x[1])
        topic_list.append(topic)
    return topic_list

# Get the dominant topic for each description
df['Topic'] = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=processed_docs)

# Save the enhanced DataFrame to a new CSV file
df.to_csv('news_articles_with_topics.csv', index=False)